**Netflix Movie Recommendation**

In [14]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [15]:
data = pd.read_csv('netflix_titles.csv')

In [16]:
data.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China","September 9, 2019",2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...
1,80117401,Movie,Jandino: Whatever it Takes,NaN,Jandino Asporaat,United Kingdom,"September 9, 2016",2016,TV-MA,94 min,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...
2,70234439,TV Show,Transformers Prime,NaN,"Peter Cullen, Sumalee Montano, Frank Welker, J...",United States,"September 8, 2018",2013,TV-Y7-FV,1 Season,Kids' TV,"With the help of three human allies, the Autob..."
3,80058654,TV Show,Transformers: Robots in Disguise,NaN,"Will Friedle, Darren Criss, Constance Zimmer, ...",United States,"September 8, 2018",2016,TV-Y7,1 Season,Kids' TV,When a prison ship crash unleashes hundreds of...
4,80125979,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,"September 8, 2017",2017,TV-14,99 min,Comedies,When nerdy high schooler Dani finally attracts...


In [17]:
movie_data = data.copy()

In [18]:
movie_data.title = movie_data.title.str.lower()
movie_data['index'] = movie_data.index
movie_data.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,index
0,81145628,Movie,norm of the north: king sized adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China","September 9, 2019",2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...,0
1,80117401,Movie,jandino: whatever it takes,NaN,Jandino Asporaat,United Kingdom,"September 9, 2016",2016,TV-MA,94 min,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...,1
2,70234439,TV Show,transformers prime,NaN,"Peter Cullen, Sumalee Montano, Frank Welker, J...",United States,"September 8, 2018",2013,TV-Y7-FV,1 Season,Kids' TV,"With the help of three human allies, the Autob...",2
3,80058654,TV Show,transformers: robots in disguise,NaN,"Will Friedle, Darren Criss, Constance Zimmer, ...",United States,"September 8, 2018",2016,TV-Y7,1 Season,Kids' TV,When a prison ship crash unleashes hundreds of...,3
4,80125979,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,"September 8, 2017",2017,TV-14,99 min,Comedies,When nerdy high schooler Dani finally attracts...,4


In [ ]:
movie_data.columns

In [19]:
features = ['director', 'cast', 'country', 'description', 'listed_in','type']

In [20]:
def combine_features(row):
    return row['director'] +" "+row['cast']+" "+row["country"]+" "+row["description"]+" "+row["listed_in"]

In [21]:
for feature in features:
    movie_data[feature] = movie_data[feature].fillna('')
    
movie_data["combined_features"] = movie_data.apply(combine_features,axis=1)
cv = CountVectorizer()
count_matrix = cv.fit_transform(movie_data["combined_features"])

In [22]:
cosine_sim = cosine_similarity(count_matrix)

In [23]:
def get_title_from_index(index):
    return movie_data[movie_data.index == index]["title"].values[0]

In [24]:
def get_title_from_index(df, index):
    return df[df.index == index]["title"].values[0]

In [25]:
def get_index_from_title(df, title):
    return df[df.title == title]["index"].values[0]

In [26]:
def recommend(movie_user_likes):
    try:
        movie_user_likes = movie_user_likes.lower()
        movie_index = get_index_from_title(movie_data, movie_user_likes)
        similar_movies =  list(enumerate(cosine_sim[movie_index]))
        sorted_similar_movies = sorted(similar_movies,key=lambda x:x[1],reverse=True)[1:]
        i=0
        print("Top 5 similar movies to "+movie_user_likes+" are:\n")
        recommended_movies = []
        for element in sorted_similar_movies:
            recommended_movies.append(get_title_from_index(movie_data, element[0]))
            i=i+1
            if i>=5:
                break
        return recommended_movies
    except:
        return('Movie not found on Netflix. Please retry!')

In [36]:
recommend('explained')

Top 5 similar movies to explained are:



['sex, explained',
 'the eighties',
 'prohibition: a film by ken burns and lynn novick',
 'woodstock',
 'bobby kennedy for president']

In [12]:
def predict():
    
    if request.method == 'POST':
        input_movie = request.form.get('movie')
    movies = recommend(input_movie)

    if type(movies)!=type('string'):
        return render_template('recommend.html',movie=input_movie,r=movies,t='s')
    else:
        return render_template('recommend.html',movie=input_movie,r=movies,t='l')